# Match contours


## Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg

from snap_fit.config.snap_fit_config import get_snap_fit_paths
from snap_fit.config.types import EDGE_ENDS_TO_CORNER, EDGE_POSS
from snap_fit.image.process import find_contours, find_corners
from snap_fit.image.utils import draw_contour, draw_corners, show_image_mpl
from snap_fit.puzzle.sheet import Sheet

In [ ]:
sf_paths = get_snap_fit_paths()
data_fol = sf_paths.data_fol
sample_fol = data_fol / "sample"

In [ ]:
img_fn = "back_03.jpg"
# img_fn = "back_04.jpg"
img_fp = sample_fol / img_fn
img_fp

In [ ]:
sheet = Sheet(img_fp)
lg.info(f"found {len(sheet.pieces)} pieces")

In [ ]:
show_image_mpl(sheet.img_orig)

In [ ]:
piece = sheet.pieces[5]

In [ ]:
img_contour = draw_contour(piece.img_bw, piece.contour_loc, color=127)
img_corners = draw_corners(img_contour, piece.corners.values(), color=190)
show_image_mpl(img_corners)

In [ ]:
piece.corners

## Split contour


### Find the corners in the contour


In [ ]:
# # contours are x, y
# con = piece.contour_loc
# lg.debug(f"contour shape: {con.shape}")
# lg.debug(f"first point: {con[0][0]}")

In [ ]:
# # subtract the corner we want to match from the contour
# corner = piece.corners["top_left"]
# con_diff = con - corner
# lg.debug(f"corner: {corner}")
# lg.debug(f"first point diff: {con_diff[0][0]}")

In [ ]:
# # find the index of the corner in the contour
# # which is the point with the smallest manhattan distance to the corner
# corner_idx = abs(con_diff).sum(axis=1).sum(axis=1).argmin()
# lg.debug(f"corner index: {corner_idx}")
# lg.debug(f"corner point: {con[corner_idx][0]}")
# lg.debug(f"corner point diff: {con_diff[corner_idx][0]}")

In [ ]:
for corner_name, corner in piece.corners.items():
    corner_coords = piece.corners[corner_name]
    cont_corner_idxs = piece.contour_corner_idxs[corner_name]
    cont_corner_coords = piece.contour_corner_coords[corner_name]
    lg.debug(
        f"{corner_name}: {corner_coords} -> {cont_corner_idxs} -> {cont_corner_coords}"
    )

### Split the contour in four segments


In [ ]:
piece.split_contour()

In [ ]:
for edge_name, edge_ends in EDGE_ENDS_TO_CORNER.items():
    start_idx = piece.contour_corner_idxs[edge_ends[0]]
    end_idx = piece.contour_corner_idxs[edge_ends[1]]
    ends_coords = piece.contour_segments_ends_coords[edge_name]
    lg.debug(f"{edge_name}: {start_idx} -> {end_idx} ({ends_coords})")

In [ ]:
tot_len = 0
for edge_name, edge_ends in EDGE_ENDS_TO_CORNER.items():
    segment = piece.contour_segments[edge_name]
    lg.debug(f"{edge_name}: {len(segment)}")
    tot_len += len(segment)
lg.debug(f"total length: {tot_len}")
lg.debug(f"total contour length: {len(piece.contour_loc)}")